In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorboardX
!pip install efficientnet_pytorch

     |████████████████████████████████| 124 kB 13.2 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=21787c76d4c118da86f5b522e285af64f139f147121c00cc2dac025a887fc762
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [3]:
import os
import shutil

shutil.copyfile('/content/drive/MyDrive/Chexpert.zip' , 'Chexpert.zip')
!unzip Chexpert.zip

Streaming output truncated to the last 5000 lines.
   creating: Chexpert/CheXpert-v1.0-small/train/patient63397/
   creating: Chexpert/CheXpert-v1.0-small/train/patient63397/study1/
 extracting: Chexpert/CheXpert-v1.0-small/train/patient63397/study1/view1_frontal.jpg  
   creating: Chexpert/CheXpert-v1.0-small/train/patient63397/study2/
 extracting: Chexpert/CheXpert-v1.0-small/train/patient63397/study2/view1_frontal.jpg  
   creating: Chexpert/CheXpert-v1.0-small/train/patient63398/
   creating: Chexpert/CheXpert-v1.0-small/train/patient63398/study1/
 extracting: Chexpert/CheXpert-v1.0-small/train/patient63398/study1/view1_frontal.jpg  
   creating: Chexpert/CheXpert-v1.0-small/train/patient63399/
   creating: Chexpert/CheXpert-v1.0-small/train/patient63399/study1/
 extracting: Chexpert/CheXpert-v1.0-small/train/patient63399/study1/view1_frontal.jpg  
   creating: Chexpert/CheXpert-v1.0-small/train/patient63400/
   creating: Chexpert/CheXpert-v1.0-small/train/patient63400/study1/
 ext

In [1]:
%cd Chexpert

/content/Chexpert


In [2]:
import torch
from PIL import Image
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
import cv2
import os
from sklearn import metrics
from PIL import Image
from data.imgaug import GetTransforms
from data.utils import transform
np.random.seed(0)
from easydict import EasyDict as edict
import json
import sys
import os
import argparse
import logging
import json
import time
import subprocess
from shutil import copyfile
from tensorboardX import SummaryWriter
import torch.nn as nn
from torch.optim import SGD, Adadelta, Adagrad, Adam, RMSprop
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn.functional as F
import tqdm

# Data Loader

In [3]:
data_transforms = transforms.Compose([
        transforms.Resize((320, 320)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
data_aug_transforms = transforms.Compose([
        transforms.Resize((320, 320)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(degrees=(-180,180)),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 2)),#new
        transforms.RandomAutocontrast(p=0.1),

        transforms.ToTensor(),
        #transforms.Normalize([0.5], [0.5])
    ])
class CheXDataset(Dataset):
    def __init__(self, csv_path, diseases,transform=None, make_one=1, mode='train'):
        self.csv_path = csv_path
        self.df = pd.read_csv(self.csv_path)
        self.df = self.df.fillna(0)
        self.make_one = make_one
        header = list(df.columns)
        self._label_header = [
                header[7],
                header[10],
                header[11],
                header[13],
                header[15]]
        self.defult_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
        self.transform = transform
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = Image.open(self.df['Path'].values[idx]).convert('RGB') 
        if self.transform is not None:
            image = self.transform(image)
        else :
            image = self.defult_transform(image)
            
  
        labels = np.zeros(len(diseases))
        
        if int(self.df['No Finding'].values[idx]) == 1  :
            labels = labels
        else:
            for i in range(len(labels)):
                if self.df[diseases[i]].values[idx] != -1 :
                    labels[i] = self.df[diseases[i]].values[idx]
                elif self.make_one :
                    labels[i] = 1
                else :
                    labels[i] = 0  
                
                
        return image,labels

In [4]:
import pandas as pd
df = pd.read_csv('config/train.csv')

In [5]:
# import pandas as pd
# df = pd.read_csv('config/train.csv')
# csv_path = 'config/train.csv'
# diseases = ['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural_Effusion']
# dataset = CheXDataset(csv_path, diseases, data_aug_transforms)

In [6]:
# a,b = dataset[0]

In [7]:
# b.shape

In [8]:
diseases = ['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural_Effusion']


# Model|

In [27]:
import torch
from torch import nn


class PcamPool(nn.Module):

    def __init__(self):
        super(PcamPool, self).__init__()

    def forward(self, feat_map, logit_map):
        assert logit_map is not None

        prob_map = torch.sigmoid(logit_map)
        weight_map = prob_map / prob_map.sum(dim=2, keepdim=True)\
            .sum(dim=3, keepdim=True)
        feat = (feat_map * weight_map).sum(dim=2, keepdim=True)\
            .sum(dim=3, keepdim=True)

        return feat


class LogSumExpPool(nn.Module):

    def __init__(self, gamma=.9):
        super(LogSumExpPool, self).__init__()
        self.gamma = gamma

    def forward(self, feat_map):
        """
        Numerically stable implementation of the operation
        Arguments:
            feat_map(Tensor): tensor with shape (N, C, H, W)
            return(Tensor): tensor with shape (N, C, 1, 1)
        """
        (N, C, H, W) = feat_map.shape

        # (N, C, 1, 1) m
        m, _ = torch.max(
            feat_map, dim=-1, keepdim=True)[0].max(dim=-2, keepdim=True)

        # (N, C, H, W) value0
        value0 = feat_map - m
        area = 1.0 / (H * W)
        g = self.gamma

        # TODO: split dim=(-1, -2) for onnx.export
        return m + 1 / g * torch.log(area * torch.sum(
            torch.exp(g * value0), dim=(-1, -2), keepdim=True))


class ExpPool(nn.Module):

    def __init__(self):
        super(ExpPool, self).__init__()

    def forward(self, feat_map):
        """
        Numerically stable implementation of the operation
        Arguments:
            feat_map(Tensor): tensor with shape (N, C, H, W)
            return(Tensor): tensor with shape (N, C, 1, 1)
        """

        EPSILON = 1e-7
        (N, C, H, W) = feat_map.shape
        m, _ = torch.max(
            feat_map, dim=-1, keepdim=True)[0].max(dim=-2, keepdim=True)

        # caculate the sum of exp(xi)
        # TODO: split dim=(-1, -2) for onnx.export
        sum_exp = torch.sum(torch.exp(feat_map - m),
                            dim=(-1, -2), keepdim=True)

        # prevent from dividing by zero
        sum_exp += EPSILON

        # caculate softmax in shape of (H,W)
        exp_weight = torch.exp(feat_map - m) / sum_exp
        weighted_value = feat_map * exp_weight

        # TODO: split dim=(-1, -2) for onnx.export
        return torch.sum(weighted_value, dim=(-1, -2), keepdim=True)


class LinearPool(nn.Module):

    def __init__(self):
        super(LinearPool, self).__init__()

    def forward(self, feat_map):
        """
        Arguments:
            feat_map(Tensor): tensor with shape (N, C, H, W)
            return(Tensor): tensor with shape (N, C, 1, 1)
        """
        EPSILON = 1e-7
        (N, C, H, W) = feat_map.shape

        # sum feat_map's last two dimention into a scalar
        # so the shape of sum_input is (N,C,1,1)
        # TODO: split dim=(-1, -2) for onnx.export
        sum_input = torch.sum(feat_map, dim=(-1, -2), keepdim=True)

        # prevent from dividing by zero
        sum_input += EPSILON

        # caculate softmax in shape of (H,W)
        linear_weight = feat_map / sum_input
        weighted_value = feat_map * linear_weight

        # TODO: split dim=(-1, -2) for onnx.export
        return torch.sum(weighted_value, dim=(-1, -2), keepdim=True)


class GlobalPool(nn.Module):

    def __init__(self, pool = 'AVG'):
        super(GlobalPool, self).__init__()
        self.pool = pool
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.maxpool = nn.AdaptiveMaxPool2d((1, 1))
        self.exp_pool = ExpPool()
        self.pcampool = PcamPool()
        self.linear_pool = LinearPool()
        self.lse_pool = LogSumExpPool()

    def cuda(self, device=None):
        return self._apply(lambda t: t.cuda(device))

    def forward(self, feat_map, logit_map):
        if self.pool == 'AVG':
            return self.avgpool(feat_map)
        elif self.pool == 'MAX':
            return self.maxpool(feat_map)
        elif self.pool == 'PCAM':
            return self.pcampool(feat_map, logit_map)
        elif self.pool == 'AVG_MAX':
            a = self.avgpool(feat_map)
            b = self.maxpool(feat_map)
            return torch.cat((a, b), 1)
        elif self.pool == 'AVG_MAX_LSE':
            a = self.avgpool(feat_map)
            b = self.maxpool(feat_map)
            c = self.lse_pool(feat_map)
            return torch.cat((a, b, c), 1)
        elif self.pool == 'EXP':
            return self.exp_pool(feat_map)
        elif self.pool == 'LINEAR':
            return pool(feat_map)
        elif self.pool == 'LSE':
            return self.lse_pool(feat_map)
        else:
            raise Exception('Unknown pooling type : {}'
                            .format(self.cfg.global_pool))
class CAModule(nn.Module):
    """
    Re-implementation of Squeeze-and-Excitation (SE) block described in:
    *Hu et al., Squeeze-and-Excitation Networks, arXiv:1709.01507*
    code reference:
    https://github.com/kobiso/CBAM-keras/blob/master/models/attention_module.py
    """

    def __init__(self, num_channels, reduc_ratio=2):
        super(CAModule, self).__init__()
        self.num_channels = num_channels
        self.reduc_ratio = reduc_ratio

        self.fc1 = nn.Linear(num_channels, num_channels // reduc_ratio,
                             bias=True)
        self.fc2 = nn.Linear(num_channels // reduc_ratio, num_channels,
                             bias=True)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, feat_map):
        # attention branch--squeeze operation
        gap_out = feat_map.view(feat_map.size()[0], self.num_channels,
                                -1).mean(dim=2)

        # attention branch--excitation operation
        fc1_out = self.relu(self.fc1(gap_out))
        fc2_out = self.sigmoid(self.fc2(fc1_out))

        # attention operation
        fc2_out = fc2_out.view(fc2_out.size()[0], fc2_out.size()[1], 1, 1)
        feat_map = torch.mul(feat_map, fc2_out)

        return feat_map


In [30]:
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 11 23:26:04 2020

@author: Ahsan
"""
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from collections import OrderedDict
import efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

__all__ = ['EfficientNet', 'EfficientNet_AG']

class Classifier_Pool(nn.Module) :
    def __init__(self, n_class) :
        super(Classifier_Pool,self).__init__()
        self.n_class = n_class
        for i in range(self.n_class):
            setattr(self,'conv_'+str(i+1), nn.Conv2d(1536,1, kernel_size=1,stride = 1, padding = 0))
            
        
        self.sigmoid = nn.Sigmoid()
        self.global_pool = GlobalPool(pool = 'AVG')
        for i in range(self.n_class):
            setattr(self,'attention_'+str(i+1), CAModule(1536))
        
    def forward(self,feat_map):
        logits = list()
        logit_maps = list()
        #attentions = list()
        # print(feat_map.shape)
        for i in range(self.n_class) :
        
            classifier = getattr(self, 'conv_' + str(i+1))
            attention = getattr(self, 'attention_'+str(i+1))
            feat_map = attention(feat_map)
            logit_map = None
            logit_map = classifier(feat_map)
            logit_maps.append(logit_map.squeeze())
            feat = self.global_pool(feat_map, logit_map)
            logit = classifier(feat)
            logit = logit.squeeze(-1).squeeze(-1)
            #feat = F.dropout(feat, p=self.fc_drop, training=self.training)
            logit = self.sigmoid(logit) 
            logits.append(logit)
            
        if logits[0].shape[0] == 1 :
            output = torch.cat(logits,dim = 0).t() 
        else :
            output = torch.cat(logits,dim = 1)                        
        return output,logit_maps
def my_efficientnet_forward(self, inputs):
    """ Calls extract_features to extract features, applies final linear layer, and returns logits. """
    bs = inputs.size(0)
    
    # Convolution layers
    x = self.extract_features(inputs)
    
    # Pooling and final linear layer
    #print(x.shape)
    
    return x

efficientnet_pytorch.EfficientNet.forward.__code__ = my_efficientnet_forward.__code__

def EfficientNet_AG(compound_coeff=0, num_classes= 1000, **kwargs):
   
    model = EfficientNet_(num_init_features=64, compound_coeff=compound_coeff, num_classes=num_classes )

    return model

class EfficientNet_(nn.Module) :
    
    def __init__(self,num_init_features=3, compound_coeff = 0, weights_path=None, num_classes=5) :
        super(EfficientNet_, self).__init__()
        
        model= EfficientNet.from_pretrained(f'efficientnet-b{compound_coeff}',in_channels=3)
        
        self.model = model
        self.model.add_module('norm5', nn.BatchNorm2d(1536))
        self.classifier = Classifier_Pool(n_class=5)
        self.Sigmoid = nn.Sigmoid()
       
    
    def forward(self, x):
#        features = self.features(x)
#        print(features.shape)
        features = self.model(x)
#        features = self.bn(features) 
        #print(features.shape)
        out = F.relu(features, inplace=True)
        #out_after_pooling = F.avg_pool2d(out, kernel_size=7, stride=7).view(features.size(0), -1)
        #print(out_after_pooling.shape)
        out = self.classifier(out)
        
        #print(out.shape)                   
        return out

In [31]:
a = torch.rand((1,3,320,320))
model = EfficientNet_AG(3,5)
b = model(a)

Loaded pretrained weights for efficientnet-b3


In [32]:
x,y = b
x.shape

torch.Size([1, 5])

# Configuration 

In [50]:
from easydict import EasyDict as edict
cfg = {
     "train_csv": "config/train.csv",
    "dev_csv": "config/dev.csv",
    "backbone": "densenet121",
    "width": 320,
    "height": 320,
    "long_side": 320,
    "fix_ratio": True,
    "pixel_mean": 128.0,
    "pixel_std": 64.0,
    "use_pixel_std": True,
    "use_equalizeHist": True,
    "use_transforms_type": "Aug",
    "gaussian_blur": 3,
    "border_pad": "pixel_mean",
    "num_classes": [1,1,1,1,1],
    "batch_weight": True,
    "enhance_index": [2,6],
    "enhance_times": 1,
    "pos_weight": [1,1,1,1,1],
    "train_batch_size": 16,
    "dev_batch_size": 1,
    "pretrained": True,
    "log_every": 50,
    "test_every": 200,
    "epoch": 30,
    "norm_type": "BatchNorm",
    "global_pool": "PCAM",
    "fc_bn": False,
    "attention_map": "None",
    "lse_gamma": 0.5,
    "fc_drop": 0,
    "optimizer": "Adam",
    "criterion": "BCE",
    "lr": 0.001,
    "lr_factor": 0.1,
    "lr_epochs": [2],
    "momentum": 0.9,
    "weight_decay": 0.0,
    "best_target": "auc",
    "save_top_k": 30,
    "save_index": [0,1,2,3,4],
    'save_path' : '/content/drive/MyDrive/Biomed/efficient net',
    'logtofile' : True,
    'resume' : False,
    'pre_train' : None,
    'verbose' :  False
}
cfg  = edict(cfg)

## Traning and Testing

In [51]:
# output = torch.rand((4,5)).ge(0.5)
# label = torch.ones((4,5))
# #label = torch.sigmoid(output[index].view(-1)).ge(0.5).float()
# #acc = (target == label).float().sum() / len(label)
# output

In [52]:
def train_epoch(summary, summary_dev, cfg, model, dataloader,
                dataloader_dev, optimizer, summary_writer, best_dict,
                dev_header):
    torch.set_grad_enabled(True)
    model.train()
    
    device = torch.device('cuda')
    steps = len(dataloader_train)
    dataiter = iter(dataloader_train)
    num_tasks = 5
    label_header = dataloader.dataset._label_header
    #num_tasks = len(cfg.num_classes)

    time_now = time.time()
    loss_sum = np.zeros(num_tasks)
    acc_sum = np.zeros(num_tasks)
    for step in tqdm.tqdm(range(steps)):
        image, target = next(dataiter)
        image = image.to(device)
        target = target.to(device)
        output,_ = model(image)
        # different number of tasks
        optimizer.zero_grad()
        loss_t = F.binary_cross_entropy(output.float(), target.float(),reduction = 'none').sum(dim = 0)
        loss_sum += np.array([i.item() for i in loss_t])
        loss =  F.binary_cross_entropy(output.float(), target.float())
        
        loss.backward()
        optimizer.step()
        label = output.ge(0.5).float()
        acc_t = (target == label).float().sum(dim=0) / (label.shape[0])
        acc_sum += np.array([i.item() for i in acc_t])
        summary['step'] += 1

        if summary['step'] % cfg.log_every == 0:
            time_spent = time.time() - time_now
            time_now = time.time()

            loss_sum /= cfg.log_every
            acc_sum /= cfg.log_every
            loss_str = ' '.join(map(lambda x: '{:.5f}'.format(x), loss_sum))
            acc_str = ' '.join(map(lambda x: '{:.3f}'.format(x), acc_sum))

            sys.stdout.write(
                '{}, Train, Epoch : {}, Step : {}, Loss : {},Acc : {}, Run Time : {:.2f} sec'
                .format(time.strftime("%Y-%m-%d %H:%M:%S"),
                        summary['epoch'] + 1, summary['step'], loss_str,
                        acc_str, time_spent))

            for t in range(num_tasks):
                summary_writer.add_scalar(
                    'train/loss_{}'.format(label_header[t]), loss_sum[t],
                    summary['step'])
                summary_writer.add_scalar(
                    'train/acc_{}'.format(label_header[t]), acc_sum[t],
                    summary['step'])

            loss_sum = np.zeros(num_tasks)
            acc_sum = np.zeros(num_tasks)

        if summary['step'] % cfg.test_every == 0:
            time_now = time.time()
            summary_dev, predlist, true_list = test_epoch(
                summary_dev, cfg, model, dataloader_dev)
            time_spent = time.time() - time_now

            auclist = []
            for i in range(len(cfg.num_classes)):
                y_pred = predlist[i]
                y_true = true_list[i]
                fpr, tpr, thresholds = metrics.roc_curve(
                    y_true, y_pred, pos_label=1)
                auc = metrics.auc(fpr, tpr)
                auclist.append(auc)
            summary_dev['auc'] = np.array(auclist)

            loss_dev_str = ' '.join(map(lambda x: '{:.5f}'.format(x),
                                        summary_dev['loss']))
            acc_dev_str = ' '.join(map(lambda x: '{:.3f}'.format(x),
                                       summary_dev['acc']))
            auc_dev_str = ' '.join(map(lambda x: '{:.3f}'.format(x),
                                       summary_dev['auc']))

            sys.stdout.write('{}, Dev, Step : {}, Loss : {}, Acc : {}, Auc : {}, Mean auc: {:.3f} Run Time : {:.2f} sec'.format(
                    time.strftime("%Y-%m-%d %H:%M:%S"),
                    summary['step'],
                    loss_dev_str,
                    acc_dev_str,
                    auc_dev_str,
                    summary_dev['auc'].mean(),
                    time_spent))

            for t in range(len(cfg.num_classes)):
                summary_writer.add_scalar(
                    'dev/loss_{}'.format(dev_header[t]),
                    summary_dev['loss'][t], summary['step'])
                summary_writer.add_scalar(
                    'dev/acc_{}'.format(dev_header[t]), summary_dev['acc'][t],
                    summary['step'])
                summary_writer.add_scalar(
                    'dev/auc_{}'.format(dev_header[t]), summary_dev['auc'][t],
                    summary['step'])

            save_best = False
            mean_acc = summary_dev['acc'][cfg.save_index].mean()
            if mean_acc >= best_dict['acc_dev_best']:
                best_dict['acc_dev_best'] = mean_acc
                if cfg.best_target == 'acc':
                    save_best = True

            mean_auc = summary_dev['auc'][cfg.save_index].mean()
            if mean_auc >= best_dict['auc_dev_best']:
                best_dict['auc_dev_best'] = mean_auc
                if cfg.best_target == 'auc':
                    save_best = True

            mean_loss = summary_dev['loss'][cfg.save_index].mean()
            if mean_loss <= best_dict['loss_dev_best']:
                best_dict['loss_dev_best'] = mean_loss
                if cfg.best_target == 'loss':
                    save_best = True

            if save_best:
                torch.save(
                    {'epoch': summary['epoch'],
                     'step': summary['step'],
                     'acc_dev_best': best_dict['acc_dev_best'],
                     'auc_dev_best': best_dict['auc_dev_best'],
                     'loss_dev_best': best_dict['loss_dev_best'],
                     'state_dict': model.state_dict()},
                    os.path.join(cfg.save_path, 'best{}.tar'.format(
                        best_dict['best_idx']))
                )
                best_dict['best_idx'] += 1
                if best_dict['best_idx'] > cfg.save_top_k:
                    best_dict['best_idx'] = 1
                logging.info(
                    '{}, Best, Step : {}, Loss : {}, Acc : {},Auc :{},'
                    'Best Auc : {:.3f}' .format(
                        time.strftime("%Y-%m-%d %H:%M:%S"),
                        summary['step'],
                        loss_dev_str,
                        acc_dev_str,
                        auc_dev_str,
                        best_dict['auc_dev_best']))
        model.train()
        torch.set_grad_enabled(True)
    summary['epoch'] += 1

    return summary, best_dict


In [53]:
def test_epoch(summary, cfg, model, dataloader):
    torch.set_grad_enabled(False)
    model.eval()

    device = torch.device('cuda')
    steps = len(dataloader)
    dataiter = iter(dataloader)
    num_tasks = len(cfg.num_classes)

    loss_sum = np.zeros(num_tasks)
    acc_sum = np.zeros(num_tasks)

    predlist = list()
    true_list = list()
    for step in range(steps):
        image, target = next(dataiter)
        image = image.to(device)
        target = target.to(device)
         
        output,_ = model(image)
        if step == 0: 
            output_t = output
            target_t = target
        else :
            output_t = torch.cat([output_t, output],dim = 0)
            target_t = torch.cat([target_t, target],dim = 0)
        # different number of tasks
        loss_t = F.binary_cross_entropy(output.float(), target.float(),reduction = 'none').sum(dim = 0)
        loss_sum += np.array([i.item() for i in loss_t])
        label = output.ge(0.5).float()
        acc_t = (target == label).float().sum(dim=0) / (label.shape[0])
        acc_sum += np.array([i.item() for i in acc_t])
    summary['loss'] = loss_sum / steps
    summary['acc'] = acc_sum / steps
    predlist = [output_t[:,i].cpu().numpy() for i in range(output_t.shape[1])]
    true_list = [target_t[:,i].cpu().numpy() for i in range(target_t.shape[1])]
    return summary, predlist, true_list

# DEBUGGING TEST EPOCH

In [54]:
# model = EfficientNet_AG(3,5)
# learning_rate = 0.001
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# Lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer , step_size = 1, gamma = .1)
# device = device = torch.device('cuda:0')
# model = model.to(device).train()
# if cfg.pre_train is not None:
#     if os.path.exists(cfg.pre_train):
#         ckpt = torch.load(cfg.pre_train, map_location=device)
#         model.module.load_state_dict(ckpt)




# dataloader_train = DataLoader(
#     CheXDataset(cfg.train_csv, diseases, data_aug_transforms),
#     batch_size=cfg.train_batch_size, num_workers=0,
#     drop_last=True, shuffle=True)
# dataloader_dev = DataLoader(
#     CheXDataset(cfg.dev_csv, diseases),
#     batch_size=cfg.dev_batch_size, num_workers=0,
#     drop_last=False, shuffle=False)
# dev_header = dataloader_dev.dataset._label_header

# summary_train = {'epoch': 0, 'step': 0}
# summary_dev = {'loss': float('inf'), 'acc': 0.0}
# summary, predlist, true_list = test_epoch(summary_dev, cfg, model, dataloader_dev)


In [55]:
# auclist = []
# for i in range(len(cfg.num_classes)):
#     y_pred = predlist[i]
#     y_true = true_list[i]
#     fpr, tpr, thresholds = metrics.roc_curve(
#         y_true, y_pred, pos_label=1)
#     auc = metrics.auc(fpr, tpr)
#     auclist.append(auc)

# Normalization and optiizer

In [56]:
def get_norm(norm_type, num_features, num_groups=32, eps=1e-5):
    if norm_type == 'BatchNorm':
        return nn.BatchNorm2d(num_features, eps=eps)
    elif norm_type == "GroupNorm":
        return nn.GroupNorm(num_groups, num_features, eps=eps)
    elif norm_type == "InstanceNorm":
        return nn.InstanceNorm2d(num_features, eps=eps,
                                 affine=True, track_running_stats=True)
    else:
        raise Exception('Unknown Norm Function : {}'.format(norm_type))


def get_optimizer(params, cfg):
    if cfg.optimizer == 'SGD':
        return SGD(params, lr=cfg.lr, momentum=cfg.momentum,
                   weight_decay=cfg.weight_decay)
    elif cfg.optimizer == 'Adadelta':
        return Adadelta(params, lr=cfg.lr, weight_decay=cfg.weight_decay)
    elif cfg.optimizer == 'Adagrad':
        return Adagrad(params, lr=cfg.lr, weight_decay=cfg.weight_decay)
    elif cfg.optimizer == 'Adam':
        return Adam(params, lr=cfg.lr, weight_decay=cfg.weight_decay)
    elif cfg.optimizer == 'RMSprop':
        return RMSprop(params, lr=cfg.lr, momentum=cfg.momentum,
                       weight_decay=cfg.weight_decay)
    else:
        raise Exception('Unknown optimizer : {}'.format(cfg.optimizer))


# Training and testing loop

In [ ]:

from model.classifier import Classifier

if not os.path.exists(cfg.save_path):
    os.mkdir(cfg.save_path)
if cfg.logtofile is True:
    logging.basicConfig(filename=cfg.save_path + '/log.txt',
                        filemode="w", level=logging.INFO)
else:
    logging.basicConfig(level=logging.INFO)





device = torch.device('cuda')

model = EfficientNet_AG(3,5)
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
Lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer , step_size = 1, gamma = .1)
model = model.to(device).train()
if cfg.pre_train is not None:
    if os.path.exists(cfg.pre_train):
        ckpt = torch.load(cfg.pre_train, map_location=device)
        model.module.load_state_dict(ckpt)




dataloader_train = DataLoader(
    CheXDataset(cfg.train_csv, diseases, data_aug_transforms),
    batch_size=cfg.train_batch_size, num_workers=0,
    drop_last=True, shuffle=True)
dataloader_dev = DataLoader(
    CheXDataset(cfg.dev_csv, diseases),
    batch_size=cfg.dev_batch_size, num_workers=0,
    drop_last=False, shuffle=False)
dev_header = dataloader_dev.dataset._label_header

summary_train = {'epoch': 0, 'step': 0}
summary_dev = {'loss': float('inf'), 'acc': 0.0}
summary_writer = SummaryWriter(cfg.save_path)
epoch_start = 0
best_dict = {
    "acc_dev_best": 0.0,
    "auc_dev_best": 0.0,
    "loss_dev_best": float('inf'),
    "fused_dev_best": 0.0,
    "best_idx": 1}

if cfg.resume:
    ckpt_path = os.path.join(cfg.save_path, 'train.tar')
    ckpt = torch.load(ckpt_path, map_location=device)
    model.module.load_state_dict(ckpt['state_dict'])
    summary_train = {'epoch': ckpt['epoch'], 'step': ckpt['step']}
    best_dict['acc_dev_best'] = ckpt['acc_dev_best']
    best_dict['loss_dev_best'] = ckpt['loss_dev_best']
    best_dict['auc_dev_best'] = ckpt['auc_dev_best']
    epoch_start = ckpt['epoch']

for epoch in range(epoch_start, cfg.epoch):
    # lr = lr_schedule(cfg.lr, cfg.lr_factor, summary_train['epoch'],
    #                  cfg.lr_epochs)
    

    summary_train, best_dict = train_epoch(
        summary_train, summary_dev, cfg, model,
        dataloader_train, dataloader_dev, optimizer,
        summary_writer, best_dict, dev_header)
    Lr_scheduler.step()
    time_now = time.time()
    summary_dev, predlist, true_list = test_epoch(
        summary_dev, cfg, model, dataloader_dev)
    time_spent = time.time() - time_now

    auclist = []
    for i in range(len(cfg.num_classes)):
        y_pred = predlist[i]
        y_true = true_list[i]
        fpr, tpr, thresholds = metrics.roc_curve(
            y_true, y_pred, pos_label=1)
        auc = metrics.auc(fpr, tpr)
        auclist.append(auc)
    summary_dev['auc'] = np.array(auclist)

    loss_dev_str = ' '.join(map(lambda x: '{:.5f}'.format(x),
                                summary_dev['loss']))
    acc_dev_str = ' '.join(map(lambda x: '{:.3f}'.format(x),
                               summary_dev['acc']))
    auc_dev_str = ' '.join(map(lambda x: '{:.3f}'.format(x),
                               summary_dev['auc']))

    logging.info(
        '{}, Dev, Step : {}, Loss : {}, Acc : {}, Auc : {},'
        'Mean auc: {:.3f} ''Run Time : {:.2f} sec' .format(
            time.strftime("%Y-%m-%d %H:%M:%S"),
            summary_train['step'],
            loss_dev_str,
            acc_dev_str,
            auc_dev_str,
            summary_dev['auc'].mean(),
            time_spent))

    for t in range(len(cfg.num_classes)):
        summary_writer.add_scalar(
            'dev/loss_{}'.format(dev_header[t]), summary_dev['loss'][t],
            summary_train['step'])
        summary_writer.add_scalar(
            'dev/acc_{}'.format(dev_header[t]), summary_dev['acc'][t],
            summary_train['step'])
        summary_writer.add_scalar(
            'dev/auc_{}'.format(dev_header[t]), summary_dev['auc'][t],
            summary_train['step'])

    save_best = False

    mean_acc = summary_dev['acc'][cfg.save_index].mean()
    if mean_acc >= best_dict['acc_dev_best']:
        best_dict['acc_dev_best'] = mean_acc
        if cfg.best_target == 'acc':
            save_best = True

    mean_auc = summary_dev['auc'][cfg.save_index].mean()
    if mean_auc >= best_dict['auc_dev_best']:
        best_dict['auc_dev_best'] = mean_auc
        if cfg.best_target == 'auc':
            save_best = True

    mean_loss = summary_dev['loss'][cfg.save_index].mean()
    if mean_loss <= best_dict['loss_dev_best']:
        best_dict['loss_dev_best'] = mean_loss
        if cfg.best_target == 'loss':
            save_best = True

    if save_best:
        torch.save(
            {'epoch': summary_train['epoch'],
             'step': summary_train['step'],
             'acc_dev_best': best_dict['acc_dev_best'],
             'auc_dev_best': best_dict['auc_dev_best'],
             'loss_dev_best': best_dict['loss_dev_best'],
             'state_dict': model.state_dict()},
            os.path.join(cfg.save_path,
                         'best{}.tar'.format(best_dict['best_idx']))
        )
        best_dict['best_idx'] += 1
        if best_dict['best_idx'] > cfg.save_top_k:
            best_dict['best_idx'] = 1
        logging.info(
            '{}, Best, Step : {}, Loss : {}, Acc : {},'
            'Auc :{},Best Auc : {:.3f}' .format(
                time.strftime("%Y-%m-%d %H:%M:%S"),
                summary_train['step'],
                loss_dev_str,
                acc_dev_str,
                auc_dev_str,
                best_dict['auc_dev_best']))
    torch.save({'epoch': summary_train['epoch'],
                'step': summary_train['step'],
                'acc_dev_best': best_dict['acc_dev_best'],
                'auc_dev_best': best_dict['auc_dev_best'],
                'loss_dev_best': best_dict['loss_dev_best'],
                'state_dict': model.state_dict()},
               os.path.join(cfg.save_path, 'train.tar'))
summary_writer.close()

Loaded pretrained weights for efficientnet-b3


  0%|          | 50/13963 [00:31<2:24:18,  1.61it/s]

2021-12-28 03:17:06, Train, Epoch : 1, Step : 50, Loss : 7.84660 9.16704 8.75501 10.15214 10.87957,Acc : 0.834 0.723 0.786 0.671 0.554, Run Time : 31.08 sec

  1%|          | 100/13963 [01:01<2:23:07,  1.61it/s]

2021-12-28 03:17:37, Train, Epoch : 1, Step : 100, Loss : 6.47248 8.86148 7.76486 9.95550 10.55679,Acc : 0.864 0.718 0.824 0.696 0.578, Run Time : 30.88 sec

...

  9%|▊         | 1199/13963 [13:10<2:17:03,  1.55it/s]

2021-12-28 03:29:46, Train, Epoch : 1, Step : 1200, Loss : 7.30246 7.79383 7.28047 9.62012 9.31397,Acc : 0.824 0.759 0.823 0.691 0.691, Run Time : 31.13 sec2021-12-28 03:29:55, Dev, Step : 1200, Loss : 0.67556 0.35776 0.32382 0.55211 0.47038, Acc : 0.709 0.833 0.859 0.658 0.761, Auc : 0.744 0.845 0.864 0.775 0.857, Mean auc: 0.817 Run Time : 8.38 sec

  9%|▉         | 1250/13963 [13:51<2:15:07,  1.57it/s]

2021-12-28 03:30:27, Train, Epoch : 1, Step : 1250, Loss : 6.79328 8.14948 7.58918 9.33183 8.86160,Acc : 0.849 0.745 0.801 0.699 0.728, Run Time : 41.08 sec

  9%|▉         | 1271/13963 [14:05<2:13:24,  1.59it/s]

In [ ]:
cfg.save_path

In [ ]:
cfg.logtofile